In [ ]:
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import cartopy #Instead of from mpl_toolkits.basemap import Basemap "conda install -c conda-forge cartopy"
from netCDF4 import Dataset #"conda install -c anaconda netcdf4"
#from scipy.io import netcdf -- Alternative to netCDF4
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr                                                    # import xarray library. Necessary for daily
from mpl_toolkits.axes_grid1 import make_axes_locatable



##PREP DATA -- Open daily wind data from ERA5. Convert hourly to daily first.


##2017
path = '.nc' #Path to wind netcdf
data = Dataset(path)


lats = data.variables['latitude'][:] #these names comes from keys
lons = data.variables['longitude'][:]
time = data.variables['time'][:]
ds_nc = xr.open_dataset(path)                        # read the file for daily
tp_daily = ds_nc.resample(time='D').mean(dim='time') # dimensions (time: 2, latitude: 721, longitude: 1440)
u = tp_daily.variables['u10'][:,:,:]
v = tp_daily.variables['v10'][:,:,:]
u = u.values #Converts from pandas datafrome to numpy array. Because hourly to daily method makes df
v = v.values #now recommend df.to_numpy() 
windspeed = np.sqrt(u**2 + v**2) #Wind speed sqrt(u^2 + v^2)
polynyawind = windspeed[:,8:16,63:78] #For area defined in FigS1
dailymeanwind2017 = polynyawind.mean(axis=(1,2))



##2018
path = '.nc' #Path to dwind netcdf
data = Dataset(path)

lats = data.variables['latitude'][:] #these names comes from keys
lons = data.variables['longitude'][:]
time = data.variables['time'][:]
ds_nc = xr.open_dataset(path)                        # read the file for daily
tp_daily = ds_nc.resample(time='D').mean(dim='time') # dimensions (time: 2, latitude: 721, longitude: 1440)
u = tp_daily.variables['u10'][:,:,:]
v = tp_daily.variables['v10'][:,:,:]
u = u.values #Converts from pandas datafrome to numpy array. Because hourly to daily method makes df
v = v.values #now recommend df.to_numpy() 
#it is 19 wide, 85long. Left to right, bottom to top for indexing
windspeed = np.sqrt(u**2 + v**2) #Wind speed sqrt(u^2 + v^2)
polynyawind = windspeed[:,8:16,63:78]
dailymeanwind2018 = polynyawind.mean(axis=(1,2))




##2019
path = '.nc' #Path to  wind netcdf
data = Dataset(path)

lats = data.variables['latitude'][:] #these names comes from keys
lons = data.variables['longitude'][:]
time = data.variables['time'][:]
ds_nc = xr.open_dataset(path)                        # read the file for daily
tp_daily = ds_nc.resample(time='D').mean(dim='time') # dimensions (time: 2, latitude: 721, longitude: 1440)
u = tp_daily.variables['u10'][:,:,:]
v = tp_daily.variables['v10'][:,:,:]
u = u.values #Converts from pandas datafrome to numpy array. Because hourly to daily method makes df
v = v.values #now recommend df.to_numpy() 
windspeed = np.sqrt(u**2 + v**2) #Wind speed sqrt(u^2 + v^2)
polynyawind = windspeed[:,8:16,63:78]
dailymeanwind2019 = polynyawind.mean(axis=(1,2))




##2020
path = '.nc' #Path to  wind netcdf
data = Dataset(path)
lats = data.variables['latitude'][:] #these names comes from keys
lons = data.variables['longitude'][:]
time = data.variables['time'][:]
ds_nc = xr.open_dataset(path)                        # read the file for daily
tp_daily = ds_nc.resample(time='D').mean(dim='time') # dimensions (time: 2, latitude: 721, longitude: 1440)
u = tp_daily.variables['u10'][:,:,:]
v = tp_daily.variables['v10'][:,:,:]
u = u.values #Converts from pandas datafrome to numpy array. Because hourly to daily method makes df
v = v.values #now recommend df.to_numpy() 
windspeed = np.sqrt(u**2 + v**2) #Wind speed sqrt(u^2 + v^2)
polynyawind = windspeed[:,8:16,63:78]
dailymeanwind2020 = polynyawind.mean(axis=(1,2))

In [ ]:
## Polynya Area

import pandas as pd


b = pd.read_csv('.txt', delimiter = ",") #Path to daily polynya area
b = b.to_numpy() 


y2017area = b[69:,5] #From 9th May
y2018area = b[31:,6]
y2019area = b[31:,7]
y2020area = b[31:,8]


#Daily means for defined study area
#wind data starts first jan, area day 1st March, hence dif indexes

dailymeanwind2017_2 = dailymeanwind2017[128:304] #From 9th of May
dailymeanwind2018_2 = dailymeanwind2018[90:304]
dailymeanwind2019_2 = dailymeanwind2019[90:304] #up to but not incl. Day 304 is 31st Oct in reg year, index 304 is next day
dailymeanwind2020_2 = dailymeanwind2020[91:305] #+1 for leap year

#Concatante

AllWind = np.concatenate([dailymeanwind2017_2,dailymeanwind2018_2,dailymeanwind2019_2,dailymeanwind2020_2])
AllArea = np.concatenate([y2017area, y2018area, y2019area, y2020area])

mask = ~np.isnan(AllWind) & ~np.isnan(AllArea)
x = AllWind[mask]
y = AllArea[mask]

plt.plot(x,y,'.')

m, b = np.polyfit(x, y, 1)
#m = slope, b=intercept


plt.plot(x, m*x + b)
plt.xlabel('Wind speed ($\mathregular{ms^{-1}}$)') 
plt.ylabel('Polynya area ($\mathregular{km^{2}}$)')

plt.savefig('.jpg',bbox_inches='tight',dpi=300) #Path to save fig